### <span style="color:#91299A">Вывод уравнений для ХКУ-приближения</span> 

<span style="color:#85c1e9">Спокойствие, только спокойствие...</span> 

![](../storage/banners/1.gif)

##### <span style="color:#2c3e50">Пересчёт</span> 

In [104]:
from sympy import *
import numpy as np

t, x1, y1, z1, x0, y0, z0, vx0, vy0, vz0, w = var("t x_1 y_1 z_1 x_0 y_0 z_0 v^x_0 v^y_0 v^z_0 w_0")
c1 = 2 * z0 + vx0/w
c2 = vz0/w
c3 = -3 * z0 - 2 * vx0/w
c4 = x0 - 2 * vz0/w
c5 = vy0/w
c6 = y0
eq = [Eq(x1, c4 - 3*c1*w*t + 2*c2*cos(w*t) - 2*c3*sin(w*t)),
      Eq(y1, c5*sin(w*t) + c6*cos(w*t)),
      Eq(z1, 2*c1 + c2*sin(w*t) + c3*cos(w*t)),]
anw = solve(eq, [vx0, vy0, vz0])
anw[vx0].simplify().simplify()

w_0*(-6*t*w_0*z_0*sin(t*w_0) + x_0*sin(t*w_0) - x_1*sin(t*w_0) - 14*z_0*cos(t*w_0) + 14*z_0 + 2*z_1*cos(t*w_0) - 2*z_1)/(3*t*w_0*sin(t*w_0) + 8*cos(t*w_0) - 8)

In [2]:
anw[vy0].simplify()

w_0*(-y_0*cos(t*w_0) + y_1)/sin(t*w_0)

In [58]:
anw[vz0].simplify().simplify()

w_0*(-3*t*w_0*z_0*cos(t*w_0) + 3*t*w_0*z_1 + 2*x_0*cos(t*w_0) - 2*x_0 - 2*x_1*cos(t*w_0) + 2*x_1 + 4*z_0*sin(t*w_0) - 4*z_1*sin(t*w_0))/(3*t*w_0*sin(t*w_0) + 8*cos(t*w_0) - 8)

##### <span style="color:#2c3e50">В статье (чуть короче запись)</span> 

In [105]:
rr = 100
rv = 0.1
params = [(t, 4000), (w, 0.0011),
          (x0, np.random.uniform(-rr, rr)), (y0, np.random.uniform(-rr, rr)), (z0, np.random.uniform(-rr, rr)),
          (x1, np.random.uniform(-rr, rr)), (y1, np.random.uniform(-rr, rr)), (z1, np.random.uniform(-rr, rr))]

den = -3*t*w*sin(t*w) - 8*cos(t*w) + 8
num1 = w * ((6*t*w*z0 + x1 - x0)*sin(t*w) + (14*z0 - 2*z1)*(cos(t*w) - 1))
num2 = w * ((3*t*w*z0 + 2*x1 - 2*x0)*cos(t*w) - 3*t*w*z1 + 2*x0 - 2*x1 + 4*(z1 - z0)*sin(t*w))
vx0_code = num1 / den
vy0_code = w * (-y0 * cos(t * w) + y1) / sin(t * w)
vz0_code =  num2 / den
print(f"Разность двух записей: Δx = {(vx0_code - anw[vx0]).subs(params)}")
print(f"Разность двух записей: Δy = {(vy0_code - anw[vy0]).subs(params)}")
print(f"Разность двух записей: Δz = {(vz0_code - anw[vz0]).subs(params)}")

Δx = -5.09379918527220E-18
Δy = 0
Δz = 5.03037725948091E-18


### <span style="color:#91299A">Алгоритм 2: адаптация метода Ньютона</span> 

<span style="color:#85c1e9">Однажды ты перестанешь идти путём тыка</span> 

![](../storage/banners/2.gif)

##### <span style="color:#2ecc71">Подумаем головой</span>
Стандартный метод Ньютона сказал бы следующее:
$$\nabla \Phi = [\frac{\partial \Phi}{\partial \upsilon^x_0},\frac{\partial \Phi}{\partial \upsilon^y_0},\frac{\partial \Phi}{\partial \upsilon^z_0}]^T$$
$$\boldsymbol{\upsilon_0}^{i+1} = \boldsymbol{\upsilon_0}^{i} - \Big(\frac{D \nabla \Phi}{D \boldsymbol{\upsilon_0}} \Big)^{-1} \nabla \Phi = \boldsymbol{\upsilon_0}^{i} -
\begin{bmatrix}
\frac{\partial^2 \Phi}{\partial \upsilon^x_0 \partial \upsilon^x_0} & \frac{\partial^2 \Phi}{\partial \upsilon^x_0 \partial \upsilon^y_0} & \frac{\partial^2 \Phi}{\partial \upsilon^x_0 \partial \upsilon^z_0}\\
\frac{\partial^2 \Phi}{\partial \upsilon^x_0 \partial \upsilon^y_0} & \frac{\partial^2 \Phi}{\partial \upsilon^y_0 \partial \upsilon^y_0} & \frac{\partial^2 \Phi}{\partial \upsilon^y_0 \partial \upsilon^z_0}\\
\frac{\partial^2 \Phi}{\partial \upsilon^x_0 \partial \upsilon^z_0} & \frac{\partial^2 \Phi}{\partial \upsilon^y_0 \partial \upsilon^z_0} & \frac{\partial^2 \Phi}{\partial \upsilon^z_0 \partial \upsilon^z_0}
\end{bmatrix} ^{-1} \nabla \Phi$$
Для оценки матрицы Грамма нужно интегрировать не $4=1+3$ а $7=1+3*2$ траекторий на каждую итерацию метода Ньютона. Ну вроде как <span style="color:#2e86c1">в 2 раза медленнее, можно же быстрее</span>.

----
##### <span style="color:#2e86c1">А теперь ещё раз как следует подумаем головой</span>
Как видно по рисунку 8, функция $\Phi$ по определению неотрицательна $\to$ решаем то задачу $$\Phi^* \to 0$$ что идентично $\Phi^* \to \min$. А вот зачем туда-обратно переделывать задачи? Иначе никак вроде.

---
Зачем я делаю вектор-целевую функцию
$$\boldsymbol{\Phi}^* = \frac{\boldsymbol{r}}{r} \Phi^*$$
Википедия гласит: Можно использовать метод Ньютона для решения систем из k уравнений, что сводится к нахождению (одновременных) нулей из k непрерывно дифференцируемых функций $f:R^k\to R$. Это эквивалентно нахождению нулей одной векторнозначной функции $F:R^k\to R^k$.

Как я понял: если аргумент есть 3-вектор, то и целевая функция должна быть 3-вектором.

Вопрос на засыпку: вообще важно в каком направлении делать вектор из $\Phi^*$? 

---
---
---
---
---
##### <span style="color:#a6acaf">Примерно на этом моменте у меня ничего не получилось:</span> 
##### <span style="color:#2c3e50">Целевая функция как вектор</span> 

In [158]:
from sympy import *

v0x, v0y, v0z = var("v^x_0 v^y_0 v^z_0")
rx = Function("r^x")(v0x, v0y, v0z)
ry = Function("r^y")(v0x, v0y, v0z)
rz = Function("r^z")(v0x, v0y, v0z)
r1 = Matrix([rx, ry, rz])
v0 = Matrix([v0x, v0y, v0z])
# Ф = Function("Ф")(v0x, v0y, v0z)
Ф = rx**2 + ry**2 + rz**2

Фvec = Ф * r1 / sqrt(rx**2 + ry**2 + rz**2)
Фvec

Matrix([
[sqrt(r^x(v^x_0, v^y_0, v^z_0)**2 + r^y(v^x_0, v^y_0, v^z_0)**2 + r^z(v^x_0, v^y_0, v^z_0)**2)*r^x(v^x_0, v^y_0, v^z_0)],
[sqrt(r^x(v^x_0, v^y_0, v^z_0)**2 + r^y(v^x_0, v^y_0, v^z_0)**2 + r^z(v^x_0, v^y_0, v^z_0)**2)*r^y(v^x_0, v^y_0, v^z_0)],
[sqrt(r^x(v^x_0, v^y_0, v^z_0)**2 + r^y(v^x_0, v^y_0, v^z_0)**2 + r^z(v^x_0, v^y_0, v^z_0)**2)*r^z(v^x_0, v^y_0, v^z_0)]])

In [163]:
J = Фvec.jacobian(v0)
J

Matrix([
[(r^x(v^x_0, v^y_0, v^z_0)*Derivative(r^x(v^x_0, v^y_0, v^z_0), v^x_0) + r^y(v^x_0, v^y_0, v^z_0)*Derivative(r^y(v^x_0, v^y_0, v^z_0), v^x_0) + r^z(v^x_0, v^y_0, v^z_0)*Derivative(r^z(v^x_0, v^y_0, v^z_0), v^x_0))*r^x(v^x_0, v^y_0, v^z_0)/sqrt(r^x(v^x_0, v^y_0, v^z_0)**2 + r^y(v^x_0, v^y_0, v^z_0)**2 + r^z(v^x_0, v^y_0, v^z_0)**2) + sqrt(r^x(v^x_0, v^y_0, v^z_0)**2 + r^y(v^x_0, v^y_0, v^z_0)**2 + r^z(v^x_0, v^y_0, v^z_0)**2)*Derivative(r^x(v^x_0, v^y_0, v^z_0), v^x_0), (r^x(v^x_0, v^y_0, v^z_0)*Derivative(r^x(v^x_0, v^y_0, v^z_0), v^y_0) + r^y(v^x_0, v^y_0, v^z_0)*Derivative(r^y(v^x_0, v^y_0, v^z_0), v^y_0) + r^z(v^x_0, v^y_0, v^z_0)*Derivative(r^z(v^x_0, v^y_0, v^z_0), v^y_0))*r^x(v^x_0, v^y_0, v^z_0)/sqrt(r^x(v^x_0, v^y_0, v^z_0)**2 + r^y(v^x_0, v^y_0, v^z_0)**2 + r^z(v^x_0, v^y_0, v^z_0)**2) + sqrt(r^x(v^x_0, v^y_0, v^z_0)**2 + r^y(v^x_0, v^y_0, v^z_0)**2 + r^z(v^x_0, v^y_0, v^z_0)**2)*Derivative(r^x(v^x_0, v^y_0, v^z_0), v^y_0), (r^x(v^x_0, v^y_0, v^z_0)*Derivative(r^x(v^x

##### <span style="color:#2c3e50">Целевая функция как скаляр</span> 

In [ ]:
Ф_v0 = Matrix([Ф.diff(v0x), Ф.diff(v0y), Ф.diff(v0z)])
Ф_v0 * Ф